In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical

# Zdefiniuj podstawową ścieżkę do danych
base_dir = 'Garbage/Garbage classification/Garbage classification'

# Funkcja do ładowania danych z pliku
def load_data(file_path):
    images = []
    labels = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split(' ')
            image_name, class_label = parts[0], int(parts[1]) - 1
            image_path = os.path.join(base_dir, image_name)
            if os.path.isfile(image_path):
                images.append(image_path)
                labels.append(class_label)
            else:
                print(f"File not found: {image_path}")
    return images, np.array(labels)

# Wczytywanie danych treningowych i walidacyjnych
train_images, train_labels = load_data(os.path.join(base_dir, 'one-indexed-files-notrash_train.txt'))
val_images, val_labels = load_data(os.path.join(base_dir, 'one-indexed-files-notrash_val.txt'))

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data augmentation configuration
train_datagen = ImageDataGenerator(
   # rescale=1./255,            # Normalize images to [0, 1]
    rotation_range=40,         # Random rotations in the range (degrees, 0 to 180)
    width_shift_range=0.2,     # Random horizontal shifts
    height_shift_range=0.2,    # Random vertical shifts
    shear_range=0.2,           # Shear transformations
    zoom_range=0.2,            # Random zoom
    horizontal_flip=True,      # Random horizontal flips
    fill_mode='nearest'        # Strategy to fill in newly created pixels
)

In [3]:
def data_generator(image_paths, labels, batch_size=32):
    num_samples = len(image_paths)
    while True:  # Loop forever so the generator never terminates
        for offset in range(0, num_samples, batch_size):
            batch_samples = image_paths[offset:offset+batch_size]
            batch_labels = labels[offset:offset+batch_size]

            images = []
            for batch_sample in batch_samples:
                img = load_img(batch_sample, target_size=(150, 150))
                img = img_to_array(img)
                img /= 255.0  # Normalizacja
                images.append(img)

            X_batch = np.array(images)
            y_batch = to_categorical(batch_labels, num_classes=6)  # Zakładając, że mamy 6 klas

            yield X_batch, y_batch
            
def augmented_data_generator(image_paths, labels, batch_size=32):
    num_samples = len(image_paths)
    while True:  # Loop forever so the generator never terminates
        for offset in range(0, num_samples, batch_size):
            batch_samples = image_paths[offset:offset + batch_size]
            batch_labels = labels[offset:offset + batch_size]

            images = []
            for batch_sample in batch_samples:
                img = load_img(batch_sample, target_size=(150, 150))
                img = img_to_array(img)
                img = train_datagen.random_transform(img)  # Apply data augmentation here
                img /= 255.0  # It's important to normalize after augmentation to keep pixel values in [0, 1]
                images.append(img)

            X_batch = np.array(images)
            y_batch = to_categorical(batch_labels, num_classes=6)  # Assuming there are 6 classes

            yield X_batch, y_batch


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitors the validation loss
    min_delta=0.001,     # The minimum amount of change to qualify as an improvement
    patience=10,         # How many epochs to wait after the last time the monitored metric improved
    verbose=1,           # To log when training is stopped
    mode='min',          # The training will stop when the quantity monitored has stopped decreasing
    restore_best_weights=True  # Restores model weights from the epoch with the best value of the monitored quantity.
)

# Przykładowa architektura modelu
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(), # 
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')  # 6 klas wyjściowych
])

optimizer = Adam(learning_rate=0.0001)  # Example learning rate

# Kompilacja modelu
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [5]:
# Ustawienie rozmiaru partii
batch_size = 32

# Inicjalizacja generatorów danych
train_generator = data_generator(train_images, train_labels, batch_size=batch_size)
validation_generator = data_generator(val_images, val_labels, batch_size=batch_size)

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 batch_normalization (BatchN  (None, 72, 72, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                        

In [6]:
history = model.fit(train_generator,
                    steps_per_epoch=len(train_images) // batch_size,
                    epochs=14, validation_data=validation_generator,
                    validation_steps=len(val_images) // batch_size,
                    callbacks=[early_stopping])

Epoch 1/14
55/55 [==============================] - 16s 271ms/step - loss: 1.5533 - accuracy: 0.4335 - val_loss: 1.7252 - val_accuracy: 0.3438
Epoch 2/14
55/55 [==============================] - 15s 273ms/step - loss: 1.1255 - accuracy: 0.5611 - val_loss: 1.6795 - val_accuracy: 0.4437
Epoch 3/14
55/55 [==============================] - 14s 259ms/step - loss: 0.9364 - accuracy: 0.6475 - val_loss: 1.5983 - val_accuracy: 0.4875
Epoch 4/14
55/55 [==============================] - 15s 276ms/step - loss: 0.8299 - accuracy: 0.6918 - val_loss: 1.5374 - val_accuracy: 0.5125
Epoch 5/14
55/55 [==============================] - 15s 277ms/step - loss: 0.7450 - accuracy: 0.7252 - val_loss: 1.4240 - val_accuracy: 0.5594
Epoch 6/14
55/55 [==============================] - 15s 271ms/step - loss: 0.6456 - accuracy: 0.7684 - val_loss: 1.3075 - val_accuracy: 0.5875
Epoch 7/14
55/55 [==============================] - 14s 263ms/step - loss: 0.5305 - accuracy: 0.8237 - val_loss: 1.2158 - val_accuracy: 0.5844